In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

housing_df= pd.read_csv('WestRoxbury.csv')

In [2]:
housing_df.shape
housing_df.head()
print(housing_df)

      TOTAL VALUE    TAX  LOT SQFT   YR BUILT  GROSS AREA   LIVING AREA  \
0            344.2  4330       9965      1880         2436         1352   
1            412.6  5190       6590      1945         3108         1976   
2            330.1  4152       7500      1890         2294         1371   
3            498.6  6272      13773      1957         5032         2608   
4            331.5  4170       5000      1910         2370         1438   
...            ...   ...        ...       ...          ...          ...   
5797         404.8  5092       6762      1938         2594         1714   
5798         407.9  5131       9408      1950         2414         1333   
5799         406.5  5113       7198      1987         2480         1674   
5800         308.7  3883       6890      1946         2000         1000   
5801         447.6  5630       7406      1950         2510         1600   

      FLOORS   ROOMS  BEDROOMS   FULL BATH  HALF BATH  KITCHEN  FIREPLACE  \
0         2.0      6  

In [3]:
housing_df.columns = [s.strip().replace(' ','_') for s in housing_df.columns]

#Descriptive Stats
print('Number of rows', len(housing_df['TOTAL_VALUE']))
print('Mean of Total_Value', housing_df['TOTAL_VALUE'].mean())
housing_df.describe()

housing_df.columns
housing_df.REMODEL = housing_df.REMODEL.astype('category')
housing_df.REMODEL.cat.categories
housing_df.REMODEL.dtype

Number of rows 5802
Mean of Total_Value 392.6857149258877


CategoricalDtype(categories=['None', 'Old', 'Recent'], ordered=False)

In [4]:
missingRows = housing_df.sample(10).index
housing_df.loc[missingRows,'BEDROOMS']= np.nan
print('no of rows with vallid bedroom after setting to nan',housing_df['BEDROOMS'].count())

reduced_df = housing_df.dropna()
print('no of rows after removing the missing values: ',len(reduced_df))

medianBedroom = housing_df['BEDROOMS'].median()
housing_df.BEDROOMS = housing_df.BEDROOMS.fillna(value=medianBedroom)
print('no of rows with vallid bedroom after filling na values',housing_df['BEDROOMS'].count())


no of rows with vallid bedroom after setting to nan 5792
no of rows after removing the missing values:  5792
no of rows with vallid bedroom after filling na values 5802


In [11]:
from sklearn import preprocessing
df = housing_df.copy()
df.head()

exclude = df.loc[:, df.columns != 'REMODEL']
exclude.head()

norm_data = preprocessing.normalize(exclude, axis=0)
norm_df = pd.DataFrame(norm_data,columns=[exclude.columns])

print('Original Data \n',exclude.head(10))
print('Normalized Data \n',norm_df.head(10))

Original Data 
    TOTAL_VALUE   TAX  LOT_SQFT  YR_BUILT  GROSS_AREA  LIVING_AREA  FLOORS  \
0        344.2  4330      9965      1880        2436         1352     2.0   
1        412.6  5190      6590      1945        3108         1976     2.0   
2        330.1  4152      7500      1890        2294         1371     2.0   
3        498.6  6272     13773      1957        5032         2608     1.0   
4        331.5  4170      5000      1910        2370         1438     2.0   
5        337.4  4244      5142      1950        2124         1060     1.0   
6        359.4  4521      5000      1954        3220         1916     2.0   
7        320.4  4030     10000      1950        2208         1200     1.0   
8        333.5  4195      6835      1958        2582         1092     1.0   
9        409.4  5150      5093      1900        4818         2992     2.0   

   ROOMS  BEDROOMS  FULL_BATH  HALF_BATH  KITCHEN  FIREPLACE  
0      6       3.0          1          1        1          0  
1     10  

In [14]:

rescale_df = (exclude-exclude.min())/(exclude.max()-exclude.min())
scaler = MinMaxScaler()
rescale = pd.DataFrame(scaler.fit_transform(exclude),index=exclude.index, columns=exclude.columns)

print('Original Data \n',exclude.head(10))
print('Rescaled Data \n',rescale.head(10))

Original Data 
    TOTAL_VALUE   TAX  LOT_SQFT  YR_BUILT  GROSS_AREA  LIVING_AREA  FLOORS  \
0        344.2  4330      9965      1880        2436         1352     2.0   
1        412.6  5190      6590      1945        3108         1976     2.0   
2        330.1  4152      7500      1890        2294         1371     2.0   
3        498.6  6272     13773      1957        5032         2608     1.0   
4        331.5  4170      5000      1910        2370         1438     2.0   
5        337.4  4244      5142      1950        2124         1060     1.0   
6        359.4  4521      5000      1954        3220         1916     2.0   
7        320.4  4030     10000      1950        2208         1200     1.0   
8        333.5  4195      6835      1958        2582         1092     1.0   
9        409.4  5150      5093      1900        4818         2992     2.0   

   ROOMS  BEDROOMS  FULL_BATH  HALF_BATH  KITCHEN  FIREPLACE  
0      6       3.0          1          1        1          0  
1     10  

In [16]:
from sklearn.model_selection import train_test_split

trainData, validData = train_test_split(housing_df, test_size=0.40)
print('Training: ', trainData.shape)
print('Validation: ', validData.shape)
print()


Training:  (3481, 14)
Validation:  (2321, 14)



In [20]:
#Fitting the regression model in traning data
from sklearn.linear_model import LinearRegression

housing_df= pd.read_csv('WestRoxbury.csv')
housing_df.columns = [s.strip().replace(' ','_') for s in housing_df.columns]
housing_df = pd.get_dummies(housing_df, prefix_sep ='_', drop_first=True)

#List of predictor and outcome
excludeColumns = ('TOTAL_VALUE','TAX')
predictors = [s for s in housing_df.columns if s not in excludeColumns]
outcome = 'TOTAL_VALUE'

X= housing_df[predictors]
y= housing_df[outcome]
train_X,valid_X,train_y,valid_y = train_test_split(X,y,test_size=0.4,random_state=1)

In [21]:
model=LinearRegression()
model.fit(train_X,train_y)


train_pred = model.predict(train_X)
train_results = pd.DataFrame({
    'TOTAL_VALUE':train_y,
    'PREDICTED': train_pred,
    'RESIDUAL': train_y-train_pred })

train_results.head()

,TOTAL_VALUE,PREDICTED,RESIDUAL
2024,392.0,387.726258,4.273742
5140,476.3,430.785540,45.514460
5259,367.4,384.042952,-16.642952
421,350.3,369.005551,-18.705551
1401,348.1,314.725722,33.374278


In [22]:
valid_pred = model.predict(valid_X)
valid_results = pd.DataFrame({
    'TOTAL_VALUE':valid_y,
    'PREDICTED': valid_pred,
    'RESIDUAL': valid_y-valid_pred })

valid_results.head()

,TOTAL_VALUE,PREDICTED,RESIDUAL
1822,462.0,406.946377,55.053623
1998,370.4,362.888928,7.511072
5126,407.4,390.287208,17.112792
808,316.1,382.470203,-66.370203
4034,393.2,434.334998,-41.134998


In [31]:
import piplite
await piplite.install('dmba')

from dmba import regressionSummary
#training
regressionSummary(train_results.TOTAL_VALUE, train_results.PREDICTED)

#Validation
regressionSummary(valid_results.TOTAL_VALUE, valid_results.PREDICTED)


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 43.0306
            Mean Absolute Error (MAE) : 32.6042
          Mean Percentage Error (MPE) : -1.1116
Mean Absolute Percentage Error (MAPE) : 8.4886

Regression statistics

                      Mean Error (ME) : -0.1463
       Root Mean Squared Error (RMSE) : 42.7292
            Mean Absolute Error (MAE) : 31.9663
          Mean Percentage Error (MPE) : -1.0884
Mean Absolute Percentage Error (MAPE) : 8.3283
